가설을 세우기 전 데이터 구조 파악 단계

가장 먼저 확인할 것
1. 각 로그 row 수 확인
2. 전체 기간 길이 (몇 개월?)
3. 사용자 수 확인 (readme: 1k users)

아래의 질문에 대한 답을 하면서 파악해나갈 것.
1. 평균 근무 시작 시간은?
2. 평균 파일 copy 수는?
3. USB 사용 분포는?
4. email 발송 분포는?
5. heavy tail 존재?

개인별 baseline 존재 여부 확인
- user별 평균
- user별 표준편차
- 전체 평균 대비 user 평균 분산

In [1]:
import pandas as pd
from pathlib import Path
from functools import reduce

DATA_PATH = Path("dataset/r4.2")

# Load each file once with only the columns needed for the whole notebook
logon = pd.read_csv(DATA_PATH / "logon.csv", usecols=["date", "user", "activity"], parse_dates=["date"])
device = pd.read_csv(DATA_PATH / "device.csv", usecols=["date", "user", "activity"], parse_dates=["date"])
file_df = pd.read_csv(DATA_PATH / "file.csv", usecols=["date", "user", "content"], parse_dates=["date"])
email = pd.read_csv(DATA_PATH / "email.csv", usecols=["date", "user"], parse_dates=["date"])
http = pd.read_csv(DATA_PATH / "http.csv", usecols=["date", "user"], parse_dates=["date"])

# Column names (from loaded dfs)
for name, df in [("logon", logon), ("device", device), ("email", email), ("file", file_df), ("http", http)]:
    print(f"{name}.csv columns: {df.columns.tolist()}")

logon.csv columns: ['date', 'user', 'activity']
device.csv columns: ['date', 'user', 'activity']
email.csv columns: ['date', 'user']
file.csv columns: ['date', 'user', 'content']
http.csv columns: ['date', 'user']


In [ ]:
# Structural checks and discovery (using loaded logon, device, file_df, email, http)

datasets = [
    ("logon.csv", logon),
    ("device.csv", device),
    ("email.csv", email),
    ("file.csv", file_df),
    ("http.csv", http),
]
all_dates = []
all_user_sets = []

print("=== 1. 각 로그 row 수 ===")
for fname, df in datasets:
    all_dates.append(df["date"])
    all_user_sets.append(set(df["user"]))
    print(f"  {fname}: {len(df):,} rows")

print("\n=== 2. 전체 기간 길이 ===")
dates_parsed = pd.to_datetime(pd.concat(all_dates, ignore_index=True))
span_days = (dates_parsed.max() - dates_parsed.min()).days
span_months = span_days / 30.44
print(f"  시작일: {dates_parsed.min()}")
print(f"  종료일: {dates_parsed.max()}")
print(f"  일수: {span_days}일 → 약 {span_months:.1f}개월")

print("\n=== 3. 사용자 수 (readme: 1k users) ===")
for (fname, df) in datasets:
    print(f"  {fname}: {df['user'].nunique()} users")
print(f"  전체(합집합): {len(reduce(set.union, all_user_sets))} users")

print("\n=== activity/content 값 (discovery) ===")
print("logon.activity:", logon["activity"].value_counts().to_dict())
print("device.activity:", device["activity"].value_counts().to_dict())
print("file.content:", file_df["content"].value_counts().to_dict())

=== 1. 각 로그 row 수 ===
  logon.csv: 854,859 rows
  device.csv: 405,380 rows
  email.csv: 2,629,979 rows
  file.csv: 445,581 rows
  http.csv: 28,434,423 rows

=== 2. 전체 기간 길이 ===
  시작일: 2010-01-02 06:49:00
  종료일: 2011-05-17 06:43:35
  일수: 499일 → 약 16.4개월

=== 3. 사용자 수 (readme: 1k users) ===
  logon.csv: 1000 users
  device.csv: 265 users
  email.csv: 1000 users
  file.csv: 264 users
  http.csv: 1000 users
  전체(합집합): 1000 users

=== activity/content 값 (discovery) ===
logon.activity: {'Logon': 470591, 'Logoff': 384268}
device.activity: {'Connect': 203339, 'Disconnect': 202041}


In [ ]:
# (logon, device, file_df, email, http are already loaded in cell 2)

logon.activity: {'Logon': 470591, 'Logoff': 384268}
device.activity: {'Connect': 203339, 'Disconnect': 202041}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- 1. 평균 근무 시작 시간 (첫 Logon per user per day) ---
logon_only = logon[logon["activity"].str.contains("Logon", case=False, na=False)].copy()
logon_only["date_only"] = logon_only["date"].dt.date
first_logon = logon_only.sort_values("date").groupby(["user", "date_only"]).first().reset_index()
first_logon["start_time_min"] = (
    first_logon["date"].dt.hour * 60 + first_logon["date"].dt.minute
)
start_min = first_logon["start_time_min"]
mean_start_min = start_min.mean()
std_start_min = start_min.std()
print("=== 1. 평균 근무 시작 시간 ===")
print(f"  평균: {mean_start_min:.1f}분 (≈ {int(mean_start_min//60)}시 {int(mean_start_min%60)}분)")
print(f"  표준편차: {std_start_min:.1f}분")
print()

# --- 2. 평균 파일 copy 수 (per user per day) ---
copy_events = file_df[file_df["content"].str.contains("Copy", case=False, na=False)].copy()
copy_events["date_only"] = copy_events["date"].dt.date
copy_per_ud = copy_events.groupby(["user", "date_only"]).size()
copy_per_user = copy_events.groupby("user").size()
mean_copy_ud = copy_per_ud.mean()
std_copy_ud = copy_per_ud.std()
mean_copy_user = copy_per_user.mean()
std_copy_user = copy_per_user.std()
print("=== 2. 평균 파일 copy 수 ===")
print("  (user·day 기준) 평균:", round(mean_copy_ud, 2), ", 표준편차:", round(std_copy_ud, 2))
print("  (user당 전체) 평균:", round(mean_copy_user, 2), ", 표준편차:", round(std_copy_user, 2))
print()

# --- 3. USB 사용 분포 ---
usb = device.copy()
usb_counts = usb.groupby("user").size()
print("=== 3. USB 사용 분포 ===")
print("  USB 이벤트 수 (user별) 평균:", round(usb_counts.mean(), 2), ", 표준편차:", round(usb_counts.std(), 2))
print("  activity 값별 건수:\n", usb["activity"].value_counts())
print()

# --- 4. email 발송 분포 ---
email["date_only"] = email["date"].dt.date
email_per_ud = email.groupby(["user", "date_only"]).size()
email_per_user = email.groupby("user").size()
print("=== 4. email 발송 분포 ===")
print("  (user·day) 평균:", round(email_per_ud.mean(), 2), ", 표준편차:", round(email_per_ud.std(), 2))
print("  (user당 전체) 평균:", round(email_per_user.mean(), 2), ", 표준편차:", round(email_per_user.std(), 2))
print()

# --- 5. heavy tail 존재? ---
# 예: user별 활동량(로그인, 파일, 이메일) 분포
act_per_user = logon_only.groupby("user").size()
print("=== 5. heavy tail 존재? ===")
print("  로그인 횟수(user별) 평균:", round(act_per_user.mean(), 2), ", 표준편차:", round(act_per_user.std(), 2))
print("  (표준편차가 평균보다 크면 heavy tail 가능)")

# --- 시각화 ---
fig, axes = plt.subplots(2, 3, figsize=(14, 9))
fig.suptitle("구조 파악: 근무 시작, 파일 copy, USB, 이메일, heavy tail", fontsize=12)

# 1. 근무 시작 시간 분포
ax = axes[0, 0]
ax.hist(first_logon["start_time_min"], bins=60, color="steelblue", edgecolor="white", alpha=0.8)
ax.axvline(mean_start_min, color="red", linestyle="--", label=f"평균 {mean_start_min:.0f}분")
ax.set_xlabel("시작 시각 (0~1440분)")
ax.set_ylabel("건수")
ax.set_title("1. 근무 시작 시간 분포")
ax.legend()

# 2. 파일 copy 수 분포 (user·day)
ax = axes[0, 1]
ax.hist(copy_per_ud.clip(upper=copy_per_ud.quantile(0.99)), bins=40, color="seagreen", edgecolor="white", alpha=0.8)
ax.axvline(mean_copy_ud, color="red", linestyle="--", label=f"평균 {mean_copy_ud:.2f}")
ax.set_xlabel("copy 수 (user·day)")
ax.set_ylabel("건수")
ax.set_title("2. 파일 copy 수 분포")
ax.legend()

# 3. USB 사용 분포 (activity별)
ax = axes[0, 2]
usb["activity"].value_counts().plot(kind="bar", ax=ax, color="coral", edgecolor="white")
ax.set_xlabel("activity")
ax.set_ylabel("건수")
ax.set_title("3. USB 사용 분포 (activity별)")
ax.tick_params(axis="x", rotation=45)

# 4. email 발송 분포 (user·day)
ax = axes[1, 0]
cap = email_per_ud.quantile(0.99)
ax.hist(email_per_ud.clip(upper=cap), bins=50, color="mediumpurple", edgecolor="white", alpha=0.8)
ax.axvline(email_per_ud.mean(), color="red", linestyle="--", label=f"평균 {email_per_ud.mean():.2f}")
ax.set_xlabel("이메일 수 (user·day)")
ax.set_ylabel("건수")
ax.set_title("4. email 발송 분포")
ax.legend()

# 5. heavy tail: user별 로그인 횟수 (로그 스케일)
ax = axes[1, 1]
counts = act_per_user.value_counts().sort_index()
counts = counts[counts.index > 0]  # avoid log(0)
ax.loglog(counts.index, counts.values, "o", markersize=3, alpha=0.7)
ax.set_xlabel("user별 로그인 횟수")
ax.set_ylabel("해당 횟수인 user 수")
ax.set_title("5. heavy tail (로그인 횟수, log-log)")

# 5 보조: user별 활동량 히스토그램
ax = axes[1, 2]
ax.hist(act_per_user.clip(upper=act_per_user.quantile(0.99)), bins=50, color="teal", edgecolor="white", alpha=0.8)
ax.axvline(act_per_user.mean(), color="red", linestyle="--", label=f"평균 {act_per_user.mean():.0f}")
ax.set_xlabel("user별 로그인 횟수")
ax.set_ylabel("user 수")
ax.set_title("5. user별 활동량 분포")
ax.legend()

plt.tight_layout()
plt.show()